In [ ]:

!pip install -q transformers datasets peft bitsandbytes accelerate evaluate trl

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


In [ ]:

prompt = "The capital of Italy is"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print(f"Token IDs: {inputs['input_ids']}")
print(f"Number of tokens: {inputs['input_ids'].shape[1]}")

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1
    )

complete_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(complete_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Token IDs: tensor([[    1,   415,  5565,   302, 10828,   349]], device='cuda:0')
Number of tokens: 6
The capital of Italy is Rome, but there are a lot more cities to explore in this beautiful country. Each city has its own charm and history that makes it unique.

Italy is known for its cuisine, architecture, fashion, art, and culture. These


In [ ]:
from datasets import load_dataset


dataset = load_dataset("imdb")

def preprocess_function(examples):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []

    for review, label in zip(examples['text'], examples['label']):
        sentiment = "positive" if label == 1 else "negative"

        prompt = f"""### Instruction:
                  Analyze the sentiment of the following movie review and respond with only one word: positive or negative.

                  ### Review:
                  {review}

                  ### Sentiment:
                  """

        prompt_tokens = tokenizer(
            prompt,
            truncation=True,
            max_length=480,
            add_special_tokens=True
        )

        response_tokens = tokenizer(
            sentiment,
            add_special_tokens=False
        )

        eos_id = tokenizer.eos_token_id

        input_ids = prompt_tokens['input_ids'] + response_tokens['input_ids'] + [eos_id]

        labels = (
            [-100] * len(prompt_tokens['input_ids']) +
            response_tokens['input_ids'] +
            [eos_id]
        )

        max_length = 512
        padding_length = max_length - len(input_ids)

        if padding_length > 0:
            input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
            labels = labels + [-100] * padding_length
            attention_mask = [1] * (max_length - padding_length) + [0] * padding_length
        else:
            input_ids = input_ids[:max_length]
            labels = labels[:max_length]
            attention_mask = [1] * max_length

        all_input_ids.append(input_ids)
        all_attention_mask.append(attention_mask)
        all_labels.append(labels)

    return {
        'input_ids': all_input_ids,
        'attention_mask': all_attention_mask,
        'labels': all_labels
    }

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names,
    desc="Tokenizing"
)

# Subset
train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(2000))
eval_dataset = tokenized_dataset['test'].shuffle(seed=42).select(range(400))

print(f"Train: {len(train_dataset)} examples")
print(f"Eval: {len(eval_dataset)} examples")

In [ ]:
test_example = {'text': ['Great movie, loved it!'], 'label': [1]}
result = preprocess_function(test_example)

tokens = tokenizer.convert_ids_to_tokens(result['input_ids'][0])
labels = result['labels'][0]

for i, (tok, lab) in enumerate(zip(tokens, labels)):
    if lab != -100 or tok == '</s>':
        decoded = tokenizer.decode([lab]) if lab != -100 else "[MASKED]"
        print(f"{tok:15} -> {decoded}")


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config)


trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Total params trainable: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"Total params: {total_params:,}")

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="./mistral-sentiment-ita",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
    warmup_steps=50,
    optim="paged_adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()
